In [1]:
%load_ext autoreload

In [2]:
%aimport model

In [3]:
import skorch

In [4]:
import numpy as np

In [5]:
import keras

Using TensorFlow backend.


In [6]:
import torch

In [7]:
torch.cuda.is_available()

True

In [8]:
dataset = 'fashion_mnist'

In [9]:
dataset_config = {
    'size': 32,
    'channels': 1,
    'classes': 10
}

In [10]:
import torchvision.datasets

In [11]:
net = skorch.NeuralNetClassifier(
    module=model.WideResNet(
        dataset,
        dataset_config['size'],
        dataset_config['channels'],
        dataset_config['classes'],
        total_block_number=24,
        widen_factor=10,
    ),
    criterion=torch.nn.CrossEntropyLoss,
    batch_size=16,
    max_epochs=10,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('progress_bar', skorch.callbacks.ProgressBar()),
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/wrn.pt", monitor=None))
    ],
    device='cuda' if torch.cuda.is_available() else 'cpu',
    lr=1e-04,
    train_split=skorch.dataset.CVSplit(5, stratified=False),
)
net.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=WideResNet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ResidualBlockGroup(
        (sequential): Sequential(
          (0): ResidualBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (dropout): Dropout(p=0.5)
            (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv_transform): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (1): ResidualBlock(
            (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1,

In [13]:
import RandomErasing.transforms

In [14]:
train_dataset = torchvision.datasets.FashionMNIST(
    "~/torchvision/datasets/FashionMNIST",
    download=True,
    train=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,)),
        RandomErasing.transforms.RandomErasing(mean=[0.4914]),
    ])
)

In [15]:
test_dataset = torchvision.datasets.FashionMNIST(
    "~/torchvision/datasets/FashionMNIST",
    download=True,
    train=False,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Pad(2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,)),
    ])
)

In [ ]:
net.fit_loop(train_dataset, None, epochs=300)